In [125]:
import random
import numpy as np  # linear algebraa
import pandas as pd  # data processing, CSV file I/O (e.g. pd.read_csv)
import os

from astropy.io import fits
from astropy.table import Table
from astropy.time import Time
from astropy.coordinates import SkyCoord, solar_system, EarthLocation, ICRS, UnitSphericalRepresentation, CartesianRepresentation
from astropy import units as u

cwd = os.getcwd()  ## current work directory

In [127]:
import barycorrpy
from barycorrpy import get_BC_vel, exposure_meter_BC_vel
from barycorrpy.utils import get_stellar_data
from astroquery.simbad import Simbad

barycorrpy.__version__

'0.4.4'

In [128]:
import pandas as pd

apero = pd.read_csv(cwd + '/APERO Object Identification - Info.csv')
apero

,OBJECT,GAIADR2ID,TWOMASSID,RV,RV_REF,TEFF,TEFF_REF,ALIASES,CHECKED,FOUND,MANUAL_EDIT,COMMENTS
0,(A) alf Lac,1.988193e+18,22311751+5016571,NaN,NaN,9200.0,HEADER[CFHT],(A) alf Lac|Gaia DR2 1988193348339562880|HGAM ...,True,True,False,Updated Gaia ID from SIMBAD
1,(A) HD 142848,4.409457e+18,15565392-0057183,NaN,NaN,9200.0,HEADER[CFHT],(A) HD 142848|GSC 05019-00390|Gaia DR1 4409456...,True,True,False,Updated Gaia ID from SIMBAD
2,(A) kap UMa,1.010771e+18,09034227+4710301,NaN,NaN,9200.0,HEADER[CFHT],(A) kap UMa|uvby98 100077327|N30 2136|TD1 1343...,True,True,False,Found aliases in SIMBAD
3,(B) Tau Sco,6.044421e+18,16355295-2812576,NaN,NaN,31400.0,HEADER[CFHT],(B) Tau Sco|2E 1632.7-2806|[B10] 4218|2E 372...,True,True,False,Updated Gaia ID from SIMBAD
4,(F) 15 Peg,1.896397e+18,21522991+2847367,NaN,NaN,6810.0,HEADER[CFHT],(F) 15 Peg|PLX 5288|WEB 19470|uvby98 100207978...,True,True,False,Updated Gaia ID from SIMBAD
...,...,...,...,...,...,...,...,...,...,...,...,...
405,xiBooA,1.237091e+18,NaN,NaN,NaN,5000.0,HEADER[CFHT],xiBooA|8pc 149.26A|* ksi Boo A|WDS J14514+1906...,True,True,False,NaN
406,xiBooB,1.237091e+18,NaN,NaN,NaN,4500.0,HEADER[CFHT],xiBooB|WDS J14514+1906B|CCDM J14513+1906B|CSI+...,True,True,False,NaN
407,XZTau,3.314313e+18,04314007+1813571,NaN,NaN,1878.0,HEADER[CFHT],XZTau|[GBA2007] 1324|Gaia DR2 3314312604605602...,True,True,False,NaN
408,zetLep,2.996172e+18,05465735-1449189,NaN,NaN,9772.0,HEADER[CFHT],zetLep|SAO 150801|HD 38678|PMC 90-93 157|* ...,True,True,False,NaN


In [129]:
import string

# define bad characters for objects (alpha numeric + "_")
BAD_OBJ_CHARS = [' '] + list(string.punctuation.replace('_', ''))


def clean_object_name(rawobjname):
    """

    Clean the object name

    :param rawobjname: str, the raw object name to clean

    """

    objectname = rawobjname.strip()
    # now remove bad characters
    for bad_char in BAD_OBJ_CHARS:
        objectname = objectname.replace(bad_char, '_')

    # make all characters upper case
    objectname = objectname.upper()

    # deal with multiple underscores in a row
    while '__' in objectname:
        objectname = objectname.replace('__', '_')

    # strip leading / trailing '_'
    objectname = objectname.strip('_')

    # return cleaned object name
    return objectname



In [130]:
import string

targets_apero = []
for target in apero['OBJECT']:
    targets_apero.append(clean_object_name(target))

for target in database['OBJNAME']:
    if target not in targets_apero:
        print(target)

NOWHERE
B_HR_5501
FAKETARGET
VENUS_SEP19_4H30HST
VENUS_SEP20_4H40HST
ENGINEERING
SATURN
TEST_TARGET
MATTTEST
17BE93_FT_1
TEST
MOON
NEPTUNE


In [131]:
import pandas as pd

database = pd.read_csv(cwd + '/full_210802_v7_obj_database.csv')
database

,Unnamed: 0,OBJNAME,OBJNAME_SOURCE,GAIADR2ID,GAIAID_SOURCE,RA_DEG,RA_SOURCE,DEC_DEG,DEC_SOURCE,PMRA,...,BPMAG_SOURCE,RPMAG,RPMAG_SOURCE,EPOCH,EPOCH_SOURCE,TEFF,TEFF_SOURCE,ALIASES,ALIASES_SOURCE,USED
0,0,16SCO,update-id-gsheet,4.348404e+18,update-id-gsheet,243.030677,database-gaia-id,-8.547582,database-gaia-id,44.125511,...,database-gaia-id,5.292628,database-gaia-id,2015.5,database-gaia-id,8800.0,HEADER[CFHT],16Sco|TD1 18951|UBV 13773|UBV M 21247|YZ 9...,database-gaia-id,1
1,1,17PEG,update-id-gsheet,2.728432e+18,update-id-gsheet,329.234762,database-gaia-id,12.076434,database-gaia-id,-27.509763,...,database-gaia-id,5.457072,database-gaia-id,2015.5,database-gaia-id,8840.0,HEADER[CFHT],17Peg|GC 30719|UBV 18771|FK5 3751|GEN# +1.00...,database-gaia-id,1
2,2,18_PUP_B,update-id-gsheet,5.725123e+18,update-id-gsheet,122.641787,database-gaia-id,-13.813938,database-gaia-id,-249.068548,...,database-gaia-id,9.769883,database-gaia-id,2015.5,database-gaia-id,3600.0,HEADER[CFHT],* 18 Pup B|GJ 297.2 B|BD-13 2420B|NLTT 190...,database-gaia-id,1
3,3,1RXSJ173353_5_165515,update-id-gsheet,4.549983e+18,update-id-gsheet,263.470967,database-gaia-id,16.919733,database-gaia-id,-122.083228,...,database-gaia-id,11.027675,database-gaia-id,2015.5,database-gaia-id,2848.0,HEADER[CFHT],1RXSJ173353.5+165515|PM J17338+1655|Karmn J173...,database-gaia-id,1
4,4,20_LMI_B,update-id-gsheet,7.465459e+17,update-id-gsheet,150.206563,database-gaia-id,31.927377,database-gaia-id,-526.829896,...,database-gaia-id,12.677678,database-gaia-id,2015.5,database-gaia-id,3286.0,HEADER[CFHT],* 20 LMi B|RX J1000.8+3155|Karmn J10008+319|WD...,database-gaia-id,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
418,418,17BE93_FT_1,input,NaN,NaN,89.771375,input,19.758139,input,0.000000,...,NaN,NaN,NaN,NaN,NaN,3000.4,input,17BE93_FT_1,NaN,1
419,419,TEST,input,NaN,NaN,67.423333,input,26.549528,input,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,input,TEST,NaN,1
420,420,MOON,input,NaN,NaN,342.566125,input,-11.146583,input,0.000000,...,NaN,NaN,NaN,NaN,NaN,6000.0,input,MOON,NaN,1
421,421,HST10,NaN,NaN,NaN,83.825875,input,-5.403722,input,0.000000,...,NaN,NaN,NaN,NaN,NaN,3000.0,HEADER[CFHT],HST10|[HC2000] 774|GMR O|owh93 10|[DRS2009] 16...,NaN,1


In [75]:
targets_apero[targets_apero == 'GL699']

array(['GL699'], dtype='<U23')

In [79]:
apero['ALIASES'].values[targets_apero == 'GL699'][0]

"Gl699|USNO 876|G 140-24|Gaia DR2 4472832130942575872|ASCC 1153178|GSC 00425-02502|LSPM J1757+0441|HIC  87937|WEB 14849|LFT 1385|GAT   12|NAME Barnard Star|WISEA J175747.94+044323.8|USNO 347|GEN# +0.00403561|UCAC4 474-068224|IRAS 17553+0438|LTT 15309|GL699|CCDM J17578+0441A|[RHG95]  2849|AC2000  146626|2MASS J17574849+0441405|V* V2500 Oph|8pc 549.01|CSV   7737|GCRV 10392|BD+04  3561a|PM J17578+0441N|NSV  9910|PLX 4098|MCC 799|Zkh 269|UCAC2  33428712|UBV   15269|JP11    18|Ci 20 1069|Karmn J17578+046|GSC 00425-00184|CSI+04-17554|PLX 4098.00|HIP  87937|LHS    57|USNO-B1.0 0946-00315199|GJ   699|NLTT 45718|StKM 2-1355|1E 1755.3+0438|TYC  425-2502-1|VVO   6|uvby98 000403561|NAME Barnard's star"

In [84]:
for target in tqdm(database['OBJNAME']):

    if target not in targets_apero:
        print(target)

    if target == 'DELBOO':
        print(targets_apero[targets_apero == target])

  0%|          | 0/423 [00:00<?, ?it/s]

['DELBOO']
NOWHERE
B_HR_5501
FAKETARGET
VENUS_SEP19_4H30HST
VENUS_SEP20_4H40HST
ENGINEERING
SATURN
TEST_TARGET
MATTTEST
17BE93_FT_1
TEST
MOON
NEPTUNE


In [104]:
from astroquery.utils.tap.core import TapPlus

# url for TAP request

URL = 'http://simbad.u-strasbg.fr:80/simbad/sim-tap'

# SQL query for TAP

QUERY = """

SELECT basic.*

FROM ident JOIN basic ON ident.oidref = basic.oid

WHERE id = '{0}';

"""

object_name = []
objname = []
objname_used = []
aliases = []
ra = []
ra_source = []
dec = []
dec_source = []
pmra = []
pmra_source = []
pmdec = []
pmdec_source = []
px = []
px_source = []
rv = []
rv_source = []
epoch = []
sp = []
sp_source = []
teff = []
teff_source = []

for target in tqdm(database['OBJNAME']):
    # set up the TapPlus class
    simbad = TapPlus(url=URL)

    # execute the job
    job = simbad.launch_job(QUERY.format(target))

    # get the results
    table = job.get_results()

    try:

        ra.append(table['ra'].data[0])
        ra_source.append(table['coo_bibcode'].data[0])

        dec.append(table['dec'].data[0])
        dec_source.append(table['coo_bibcode'].data[0])

        px.append(table['plx_value'].data[0])
        px_source.append(table['plx_bibcode'].data[0])

        rv.append(table['rvz_radvel'].data[0])
        rv_source.append(table['rvz_bibcode'].data[0])

        sp.append(table['sp_type'].data[0])
        sp_source.append(table['sp_bibcode'].data[0])

        epoch.append(2451545.0)

        teff.append(database['TEFF'][database['OBJNAME'] == target].values[0])
        teff_source.append(database['TEFF_SOURCE'][database['OBJNAME'] == target].values[0])

        objname.append(target)
        objname_used.append(target)

        aliases.append(apero['ALIASES'].values[targets_apero == target][0])
        object_name.append(apero['OBJECT'].values[targets_apero == target][0])

    except IndexError:

        aliases_names = database['ALIASES'][database['OBJNAME'] == target].values[0].split('|')
        for alias in (aliases_names):
            # execute the job

            job = simbad.launch_job(QUERY.format(alias))

            # get the results

            table = job.get_results()

            try:
                ra.append(table['ra'].data[0])
                ra_source.append(table['coo_bibcode'].data[0])

                dec.append(table['dec'].data[0])
                dec_source.append(table['coo_bibcode'].data[0])

                pmra.append(table['pmra'].data[0])
                pmra_source.append(table['pm_bibcode'].data[0])

                pmdec.append(table['pmdec'].data[0])
                pmdec_source.append(table['pm_bibcode'].data[0])

                px.append(table['plx_value'].data[0])
                px_source.append(table['plx_bibcode'].data[0])

                rv.append(table['rvz_radvel'].data[0])
                rv_source.append(table['rvz_bibcode'].data[0])

                sp.append(table['sp_type'].data[0])
                sp_source.append(table['sp_bibcode'].data[0])

                epoch.append(2451545.0)

                teff.append(database['TEFF'][database['OBJNAME'] == target].values[0])
                teff_source.append(database['TEFF_SOURCE'][database['OBJNAME'] == target].values[0])

                objname.append(target)
                objname_used.append(alias)

                aliases.append(apero['ALIASES'].values[targets_apero == target][0])
                object_name.append(apero['OBJECT'].values[targets_apero == target][0])

                break
            except IndexError:
                pass

  0%|          | 0/423 [00:00<?, ?it/s]

Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
C

Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
C

Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
C

Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
C

Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
C

Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
Created TAP+ (v20200428.1) - Connection:
	Host: simbad.u-strasbg.fr
	Use HTTPS: False
	Port: 80
	SSL Port: 443
C

In [107]:
len(ra)

407

In [110]:
df = dict()
new_database = pd.DataFrame(df)

new_database['OBJECT'] = object_name
new_database['OBJNAME'] = objname
new_database['OBJNAME SOURCE'] = objname_used
new_database['ALIAS'] = aliases
new_database['RA'] = ra
new_database['RA_SOURCE'] = ra_source
new_database['DEC'] = dec
new_database['DEC_SOURCE'] = dec_source
new_database['EPOCH'] = epoch
new_database['PMRA'] = pmra
new_database['PMRA_SOURCE'] = pmra_source
new_database['PMDEC'] = pmdec
new_database['PMDEC_SOURCE'] = pmdec_source
new_database['PLX'] = px
new_database['PLX_SOURCE'] = px_source
new_database['RV'] = rv
new_database['RV_SOURCE'] = rv_source
new_database['SP_TYPE'] = sp
new_database['SP_SOURCE'] = sp_source
new_database['TEFF'] = teff
new_database['TEFF_SOURCE'] = teff_source
new_database.to_csv('new_database_test.csv')
new_database

,OBJECT,OBJNAME,OBJNAME SOURCE,ALIAS,RA,RA_SOURCE,DEC,DEC_SOURCE,EPOCH,PMRA,...,PMDEC,PMDEC_SOURCE,PLX,PLX_SOURCE,RV,RV_SOURCE,SP_TYPE,SP_SOURCE,TEFF,TEFF_SOURCE
0,16Sco,16SCO,16SCO,16Sco|TD1 18951|UBV 13773|UBV M 21247|YZ 9...,243.030485,2018yCat.1345....0G,-8.547581,2018yCat.1345....0G,2451545.0,44.126,...,-0.078,2018yCat.1345....0G,12.6484,2018yCat.1345....0G,5.0,2006AstL...32..759G,A2IV,1999MSS...C05....0H,8800.0,HEADER[CFHT]
1,17Peg,17PEG,17PEG,17Peg|GC 30719|UBV 18771|FK5 3751|GEN# +1.00...,329.234883,2018yCat.1345....0G,12.076489,2018yCat.1345....0G,2451545.0,-27.51,...,-12.816,2018yCat.1345....0G,7.5832,2018yCat.1345....0G,10.3,2006AstL...32..759G,A2Vnn,,8840.0,HEADER[CFHT]
2,* 18 Pup B,18_PUP_B,18_PUP_B,* 18 Pup B|GJ 297.2 B|BD-13 2420B|NLTT 190...,122.642892,2018yCat.1345....0G,-13.814203,2018yCat.1345....0G,2451545.0,-249.069,...,61.643,2018yCat.1345....0G,47.8089,2018yCat.1345....0G,33.17,2018MNRAS.475.1960F,M2.5V,2015A&A...577A.128A,3600.0,HEADER[CFHT]
3,1RXSJ173353.5+165515,1RXSJ173353_5_165515,1RXSJ173353.5+165515,1RXSJ173353.5+165515|PM J17338+1655|Karmn J173...,263.471516,2018yCat.1345....0G,16.920232,2018yCat.1345....0G,2451545.0,-122.083,...,-116.003,2018yCat.1345....0G,62.3715,2018yCat.1345....0G,-19.0,2014AJ....147...20N,M6V,2014AJ....147...20N,2848.0,HEADER[CFHT]
4,* 20 LMi B,20_LMI_B,20_LMI_B,* 20 LMi B|RX J1000.8+3155|Karmn J10008+319|WD...,150.209236,2018yCat.1345....0G,31.929235,2018yCat.1345....0G,2451545.0,-526.83,...,-431.407,2018yCat.1345....0G,68.8826,2018yCat.1345....0G,56.0,2015ApJ...802L..10T,M6.0V,2015A&A...577A.128A,3286.0,HEADER[CFHT]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
402,HLTau,HLTAU,[GMM2009] Tau L1551 2,HLTau|[GMM2009] Tau L1551 2|Haro 6-14|AAVSO 0...,67.910300,2012yCat.1322....0Z,18.232801,2012yCat.1322....0Z,2451545.0,4.2,...,-15.3,2012yCat.1322....0Z,--,,--,,K5,2004ApJ...616..998W,4450.0,HEADER[CFHT]
403,G055.3704+00.1858,G055_3704_00_1858,AKARI-FIS-V1 J1933238+195655,G055.3704+00.1858|AKARI-FIS-V1 J1933238+195655...,293.351037,2003yCat.2246....0C,19.948792,2003yCat.2246....0C,2451545.0,--,...,--,,--,,--,,,,5000.0,HEADER[CFHT]
404,GY92-214,GY92_214,GY92_214,GY92-214|WL 15|GBS-VLA J162709.41-243719.0|[MA...,246.789293,2003yCat.2246....0C,-24.621881,2003yCat.2246....0C,2451545.0,--,...,--,,--,,--,,,,5000.0,HEADER[CFHT]
405,GY92-378,GY92_378,GY92_378,GY92-378|GBS-VLA J162751.80-243145.9|BKLT J162...,246.965853,2003yCat.2246....0C,-24.529314,2003yCat.2246....0C,2451545.0,--,...,--,,--,,--,,,,5000.0,HEADER[CFHT]


In [122]:
df = dict()
rejected_list = pd.DataFrame(df)
rejected = []
rejected_alias = []
for value in database['OBJNAME']:
    if (value not in objname or value in others) and 'TOI' not in value:
        rejected.append(value)
        rejected_alias.append(database['ALIASES'].values[database['OBJNAME'] == value][0])
rejected_list['TARGET'] = rejected
rejected_list['ALIAS'] = rejected_alias
rejected_list.to_csv('rejected_list.csv')
rejected_list

,TARGET,ALIAS
0,M1,m1|NRAO 214|1M 0531+219|2E 0531.5+2159|BD+19 ...
1,NOWHERE,NOWHERE
2,B_HR_5501,B_HR_5501
3,FAKETARGET,FAKETARGET
4,VENUS_SEP19_4H30HST,VENUS_SEP19_4H30HST
5,VENUS_SEP20_4H40HST,VENUS_SEP20_4H40HST
6,ENGINEERING,ENGINEERING
7,G055_3704_00_1858,G055.3704+00.1858|AKARI-FIS-V1 J1933238+195655...
8,GY92_214,GY92-214|WL 15|GBS-VLA J162709.41-243719.0|[MA...
9,GY92_378,GY92-378|GBS-VLA J162751.80-243145.9|BKLT J162...
